In [1]:
# IMPORTING DEPENDENCIES
import numpy as np
LENGTH = 3

In [2]:
# CODING THE ENVIRONMENT
class Environment:
    def __init__(self):
        self.board = np.zeros((LENGTH,LENGTH))
        self.x = -1
        self.o = 1
        self.winner = None
        self.ended = False
        self.num_states = 3 ** (LENGTH*LENGTH)
        
    def is_empty(self,i,j):
        return self.board[i,j] == 0
    
    def reward(self,sym):
        # no reward until game is over
            if not self.game_over():
              return 0

            # if we get here, game is over
            # sym will be self.x or self.o
            return 1 if self.winner == sym else 0
        
    def get_state(self):
        h = 0
        k = 0
        
        for i in range(LENGTH):
            for j in range(LENGTH):
                if self.board[i,j] == 0:
                    v = 0
                elif self.board[i,j] == self.x:
                    v = 1
                elif self.board[i,j] == self.o:
                    v = 2
                    
                h = h + (3 ** k) * v
                k = k + 1
        return h
                
    def game_over(self,force_recalculate = False):
        if not force_recalculate and self.ended:
            return self.ended
        
        # CHECK FOR SUM IN ROW
        for i in range(LENGTH):
            for j in (self.x,self.o):
                if self.board[i].sum() == j * LENGTH:
                    self.winner = j
                    self.ended = True
                    return True
        
        # CHECK IF THE COLUMN IS SUM
        for i in range(LENGTH):
            for j in (self.x,self.o):
                if self.board[:,i].sum() == j * LENGTH:
                    self.winner = j
                    self.ended = True
                    return True
        
        # CHECK FOR DIAGONALS
        for player in (self.x,self.o):
            # MAIN DIAGONAL
            if self.board.trace() == player * LENGTH :
                self.winner = player
                self.ended = True
                return True
                
            # ANTI - DIAGONAL
            if np.fliplr(self.board).trace() == player * LENGTH:
                self.winner = player
                self.ended = True
                return True
            
        # CHECK FOR A DRAW
        if np.all((self.board == 0) == False) :
            self.winner = None
            self.ended = True
            return True
        
        # GAME IS NOT OVER YET
        self.winner = None
        return False
    
    def draw_board(self):
        for i in range(LENGTH):
            print("------------\n",end = '')
            for j in range(LENGTH):
                print("  ",end='')
                if self.board[i,j] == self.x:
                    print("x   |",end = '')
                elif self.board[i,j] == self.o:
                    print("o   |",end = '')
                else:
                    print("__  |",end='')
            print("\n")
        print("------------",end='')

In [3]:
class Agent:
    def  __init__(self, eps=0.1, alpha=0.5):
        self.eps = eps # probability of choosing random action instead of greedy
        self.alpha = alpha # learning rate
        self.verbose = False
        self.state_history = []
  
  def setV(self, V):
    self.V = V

  def set_symbol(self, sym):
    self.sym = sym

  def set_verbose(self, v):
    # if true, will print values for each position on the board
    self.verbose = v

  def reset_history(self):
    self.state_history = []

  def take_action(self, env):
    # choose an action based on epsilon-greedy strategy
    r = np.random.rand()
    best_state = None
    if r < self.eps:
      # take a random action
      if self.verbose:
        print("Taking a random action")

      possible_moves = []
      for i in range(LENGTH):
        for j in range(LENGTH):
          if env.is_empty(i, j):
            possible_moves.append((i, j))
      idx = np.random.choice(len(possible_moves))
      next_move = possible_moves[idx]
    else:
      # choose the best action based on current values of states
      # loop through all possible moves, get their values
      # keep track of the best value
      pos2value = {} # for debugging
      next_move = None
      best_value = -1
      for i in range(LENGTH):
        for j in range(LENGTH):
          if env.is_empty(i, j):
            # what is the state if we made this move?
            env.board[i,j] = self.sym
            state = env.get_state()
            env.board[i,j] = 0 # don't forget to change it back!
            pos2value[(i,j)] = self.V[state]
            if self.V[state] > best_value:
              best_value = self.V[state]
              best_state = state
              next_move = (i, j)

      # if verbose, draw the board w/ the values
      if self.verbose:
        print("Taking a greedy action")
        for i in range(LENGTH):
          print("------------------")
          for j in range(LENGTH):
            if env.is_empty(i, j):
              # print the value
              print(" %.2f|" % pos2value[(i,j)], end="")
            else:
              print("  ", end="")
              if env.board[i,j] == env.x:
                print("x  |", end="")
              elif env.board[i,j] == env.o:
                print("o  |", end="")
              else:
                print("   |", end="")
          print("")
        print("------------------")

    # make the move
    env.board[next_move[0], next_move[1]] = self.sym

  def update_state_history(self, s):
    # cannot put this in take_action, because take_action only happens
    # once every other iteration for each player
    # state history needs to be updated every iteration
    # s = env.get_state() # don't want to do this twice so pass it in
    self.state_history.append(s)

  def update(self, env):
    reward = env.reward(self.sym)
    target = reward
    for prev in reversed(self.state_history):
      value = self.V[prev] + self.alpha*(target - self.V[prev])
      self.V[prev] = value
      target = value
    self.reset_history()

In [4]:
class Human:
    def __init__(self):
        pass
    
    def set_symbol(self,sym):
        self.symbol = sym
        
    def take_action(self,env):
        while True:
            # BREAK UNDER A VALID MOVE
            move = input("Enter the coordinates for your move: (i,j = 0..2)")
            i,j = move.split(',')
            i = int(i)
            j = int(j)
            
            if env.isEmpty(i,j) :
                env.board[i,j] = self.symbol
                break
                
    def update_state_history(self,s):
        pass
    
    def update(self,env):
        pass

In [5]:
def get_state_hash_and_winner(env,i=0,j=0):
    result = []
    
    for v in (0,env.x,env.o):
        env.board[i,j] = v
        
        if j == 2:
            if i == 2:
                state = env.get_state()
                ended = env.game_over(force_recalculate=True)
                winner = env.winner
                
                result.append((state,winner,ended))
            else :
                result += get_state_hash_and_winner(env,i+1,0)
        else:
            result += get_state_hash_and_winner(env,i,j+1)
    return result

In [6]:
# INITIALIZING THE VALUE ARRAY AT THE START
def initialV_x(env, state_winner_triples) :
    v = np.zeros(env.num_states)
    for state,winner,ended in state_winner_triples:
        if ended :
            if winner == env.x :
                v[state] = 1
            else :
                v[state] = 0
        else :
             v[state] = 0.5
    
    return v

In [7]:
def initialV_o(env, state_winner_triples) :
    v = np.zeros(env.num_states)
    for state,winner,ended in state_winner_triples :
        if ended :
            if winner == env.o :
                v[state] = 1
            else :
                v[state] = 0
        else :
            v[state] = 0.5
            
    return v

In [8]:
def play_game(p1,p2,env,draw = False) :
    # LOOP UNTIL THE GAME IS OVER
    current_player = None
    while not env.game_over():
        # SWITCH PLACES
        if current_player == p1 :
            current_player = p2
        else :
            current_player = p1
            
        # DRAWING THE BOARD FOR THE PLAYER PLAYING
        if draw :
            if draw == 1 and current_player == p1:
                env.draw_board()
            if draw == 2 and current_player == p2:
                env.draw_board()
                
        # Current player takes his turn
        current_player.take_action(env)
        
        # State histories get updated
        state = env.get_state()
        p1.update_state_history(state)
        p2.update_state_history(state)
        
        
    if draw :
        env.draw_board()
        
    p1.update(env)
    p2.update(env)

In [ ]:
if __name__ == '__main__':
    p1 = Agent()
    p2 = Agent()

    env = Environment()
    state_winner_triples = get_state_hash_and_winner(env)

    vx = initialV_x(env,state_winner_triples)
    p1.setV(vx)

    vo = initialV_o(env,state_winner_triples)
    p2.setV(vo)

    p1.set_symbol(env.x)
    p2.set_symbol(env.o)

    # AGENT 1 AND 2 PLAY WITH EACH OTHER 10,000 TIMES
    # TO LEARN 
    
    T = 10000
    for i in range(T):
        if i%100 == 0:
            print(i)
            
        play_game(p1,p2,Environment())

    human = Human()
    human.set_symbol(env.o)
    
    while True :
        p1.set_verbose(True)

        play_game(p1,human,Environment(),draw=2)
        answer = input("Play again? [Y/n]: ")
        if answer and answer.lower()[0] == 'n':
            break

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
Taking a greedy action
------------------
 0.61| 0.55| 0.68|
------------------
 0.62| 0.88| 0.66|
------------------
 0.51| 0.43| 0.68|
------------------
------------
  __  |  __  |  __  |

------------
  __  |  x   |  __  |

------------
  __  |  __  |  __  |

------------